In [ ]:
import cv2
import numpy as np

# Initialize video capture
cap = cv2.VideoCapture('C:\\Users\\SRIRAM\\Downloads\\istockphoto-1423119278-640_adpp_is.mp4')

# Background subtractor for density
back_sub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50)

# For optical flow visualization
hsv = None
density_threshold = 40.0       # Threshold for static crowd density
motion_threshold = 2.0         # Threshold for average movement (optical flow)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if hsv is None:
        hsv = np.zeros_like(frame)
        hsv[..., 1] = 255

    # Compute dense optical flow
    if 'prev_gray' in locals():
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray,
                                            None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        cv2.imshow('Optical Flow', flow_rgb)

        # 💨 Compute motion intensity
        motion_density = np.mean(mag)
        cv2.putText(frame, f'Motion Intensity: {motion_density:.2f}', (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        if motion_density > motion_threshold:
            cv2.putText(frame, '⚠️ High Movement Detected!', (10, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            print(f'ALERT: Motion intensity exceeded! Current: {motion_density:.2f}')

    # 🧍 Background subtraction for crowd density
    fg_mask = back_sub.apply(gray)

    # Morphological filtering
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)

    # 📏 Compute crowd density
    density = np.sum(fg_mask == 255) / (fg_mask.shape[0] * fg_mask.shape[1]) * 100
    cv2.putText(frame, f'Density: {density:.2f} %', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    if density > density_threshold:
        cv2.putText(frame, '⚠️ High Crowd Density!', (10, 140),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        print(f'ALERT: Crowd density exceeded! Current: {density:.2f}%')

    # Display outputs
    cv2.imshow('Crowd Frame', frame)
    cv2.imshow('Foreground Mask', fg_mask)

    prev_gray = gray.copy()
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
